# Regressão

A tarefa de **regressão** consiste em predizer **valores** para determinados **objetos**. Vamos contextualizar de maneira operacional esses objetos e valores através de seus tipos de variáveis: em geral um valor é um número real <code>(float)</code> ou um inteiro <code>(int)</code>; já o objeto a ser regredido pode ser um **vetor de features** - aqui vetor tem o mesmo sentido de uma variável lista <code>(lst)</code> ou uma tupla <code>(tuple)</code>. 

Diferentes **algoritmos de regressão** podem ser usados para realizar essa tarefa de, dada uma entrada, gerar uma saída (valor predito para aquele objeto).

Define-se a regressão como uma tarefa de **aprendizado de máquina supervisionado**, isto é, os algoritmos de regressão precisam ser treinados com pares objeto-valor tidos como associações corretas. Somente após o treinamento é que o algoritmo de regressão estará apto a predizer valores para novas entradas (das quais não se sabe o valor a princípio) - diz-se que  algoritmo aprendeu o padrão nos dados de treinamento e agora pode ser usado para predizr o valor de saída para novas entradas.

-----------------------------------------------

## 1 - IArpi Data Set

#### Descrição geral:
Três diferentes objetos são postos a se mover em um plano inclinado devido a ação da gravidade. Atributos cinemáticos do movimento dos corpos são coletados. Pretende-se estabelecer uma relação entre esses atributos e cada tipo de objeto.

#### Objetivo:
O problema consiste na predição da velocidade média de três objetos (esfera, cilindro e aro) a partir da sua altura inicial em um plano inclinado a um determinado ângulo. O objetivo é introduzir técnicas de IA para cursos de graduação de física onde o experimento do plano inclinado é amplamente estudado (de maneira teórica e em laboratório).


#### Features (variáveis de entrada):
As features foram determinadas experimentalmente:
- Ângulo: ângulo de inclinação do plano [°]
- Altura: de partida do objeto [m]
- Tipo de objeto (esfera, cilindro, aro)

#### Alvo (valor de saída):
- Velocidade Média: velocidade média determinada pela distância/tempo [m/s]

#### Referências:
- https://github.com/simcomat/IArpi
- Ferreira, H., Almeida Junior, E. F., Espinosa-García, W., Novais, E., Rodrigues, J. N. B., & Dalpian, G. M. (2022). Introduzindo aprendizado de máquina em cursos de física: o caso do rolamento no plano inclinado. In Revista Brasileira de Ensino de Física (Vol. 44). FapUNIFESP (SciELO). https://doi.org/10.1590/1806-9126-rbef-2022-0214 

In [18]:
import pandas as pd # Para trabalhar com dados na forma de tabela

from sklearn.model_selection import train_test_split   # Separação treino/teste
from sklearn.preprocessing import MinMaxScaler         # Escalonador

# Algoritmos de Regressão
from sklearn.linear_model import LinearRegression      # Regressao Linear
from sklearn.svm import SVR                            # Regressão por Máquina de Vetor Suporte
from sklearn.tree import DecisionTreeRegressor         # Regressão por Árvore de Decisão
from sklearn.neighbors import KNeighborsRegressor      # k-vizinhos mais próximos (KNN)
from sklearn.ensemble import RandomForestRegressor     # RandomForest
from sklearn.ensemble import GradientBoostingRegressor # GradientBoosting
from sklearn.neural_network import MLPRegressor        # Multilayer Perceptron


# Módulos para plotar gráficos e ajustar formatação dos mesmos
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.ticker import AutoMinorLocator
from matplotlib.lines import Line2D
import seaborn as sns

### 1 - Primeiro passo: carregando os dados

In [10]:
tabela_dados = pd.read_csv('https://raw.githubusercontent.com/simcomat/IArpi/main/datasets/rolling.csv', sep=';') 

In [11]:
tabela_dados.head()

,Objeto,Ângulo (°),Distancia (m),Altura (m),Tempo (s),Velocidade Média (m/s)
0,esfera,7.73,1.0,0.134505,1.33,0.751880
1,esfera,7.73,1.0,0.134505,1.49,0.671141
2,esfera,7.73,1.0,0.134505,1.49,0.671141
3,esfera,7.73,1.0,0.134505,1.40,0.714286
4,esfera,7.73,1.0,0.134505,1.31,0.763359


In [12]:
tabela_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803 entries, 0 to 802
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Objeto                  803 non-null    object 
 1   Ângulo (°)              803 non-null    float64
 2   Distancia (m)           803 non-null    float64
 3   Altura (m)              803 non-null    float64
 4   Tempo (s)               803 non-null    float64
 5   Velocidade Média (m/s)  803 non-null    float64
dtypes: float64(5), object(1)
memory usage: 37.8+ KB


### 1 - Segundo passo: separação de dados

Os algoritmos de regressão não estão preparados para receber um nome (string) como entrada. Então, apenas fornecer o nome do objeto (esfera, cilindro, aro) para o modelo não irá funcionar. Precisamos primeiro converter esses nomes em uma representação numérica. Para isso vamos usar o **OneHot Encoding**. Nessa representação, cada valor do atributo objeto torna-se uma característica única (feature). Para 3 valores de objetos teremos 3 novas colunas com valores binários (verdadeiro ou falso, 1 ou 0) representando cada objeto, por exemplo, se seguirmos (esfera, cilindro, aro) uma esfera será representada pela tupla (1,0,0) enquanto um aro será por (0,0,1). Fisicamente, misturas não seriam possiveis como uma esfera-aro (1,0,1), mas em outras situações essa técnica pode ser usada para representar a presença de mais um objeto (como duas palavras diferentes em uma mesma frase).

In [13]:
ohe = pd.get_dummies(tabela_dados.Objeto, prefix='objeto') # One hot encoding
tabela_dados = tabela_dados.join(ohe)

In [14]:
tabela_dados.head()

,Objeto,Ângulo (°),Distancia (m),Altura (m),Tempo (s),Velocidade Média (m/s),objeto_aro,objeto_cilindro,objeto_esfera
0,esfera,7.73,1.0,0.134505,1.33,0.751880,0,0,1
1,esfera,7.73,1.0,0.134505,1.49,0.671141,0,0,1
2,esfera,7.73,1.0,0.134505,1.49,0.671141,0,0,1
3,esfera,7.73,1.0,0.134505,1.40,0.714286,0,0,1
4,esfera,7.73,1.0,0.134505,1.31,0.763359,0,0,1


In [17]:
x = tabela_dados[['Altura (m)','Ângulo (°)', 'objeto_aro', 'objeto_cilindro','objeto_esfera']] 
y = tabela_dados['Velocidade Média (m/s)'] # Atributo alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

### 1 - Terceiro passo: transformação dos dados

**OBS:** perceba que nem sempre a transformação ocorre apenas no terceiro passo. Tivemos que fazer uma transformação usando o One Hot Encoding (OHE) no segundo passo. A troca da ordem neste caso foi feita para facilitar a etapa de separação (não ter que filtrar a coluna objeto após transformar via OHE.

In [19]:
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### 1 - Quarto passo: treinando os algoritmos de regressão

In [20]:
# Regressao Linear
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)

# KNN Regressor
knnr = KNeighborsRegressor()
knnr.fit(x_train_scaled,y_train)

# SVM
svmr = SVR()
svmr.fit(x_train_scaled,y_train)

# Regressão por Árvore de Decisão
dtr = DecisionTreeRegressor()
dtr.fit(x_train_scaled,y_train)

# Regressão por Random
rfr = RandomForestRegressor(random_state=42)
rfr.fit(x_train_scaled, y_train)

# Regressõ por GB
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(x_train_scaled, y_train)

# Multilayer Perceptron
mlpr =  MLPRegressor(random_state=42)
mlpr.fit(x_train_scaled,y_train)

regressores = {
    'LR':lr,
    'KNNR':knnr,
    'SVMR':svmr,
    'RFR':rfr,
    'GBR':gbr,
    'MLPR':mlpr,
}

### 1 - Quinto passo: avaliar o desempenho

In [21]:
from sklearn.metrics import mean_absolute_error, r2_score       # Métricas de Regressão

In [ ]:
resultados={}
resultados_MAE={}
resultados_R2={}

for rg_name, rg in regressores.items():
    
    y_pred = rg.predict(x_test_scaled)        # Entrando os dados de teste no modelo ML
    mae = mean_absolute_error(y_test, y_pred) # Calculando métrica MAE
    r2 = r2_score(y_test, y_pred)             # Calculando métrica R2
    
    # Salvando resultados
    scoring = {'MAE': mae,
               'R2' :r2
         }
    resultados[rg_name]=scoring
    resultados_MAE[rg_name]=mae
    resultados_R2[rg_name]=r2